# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [2]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [51]:
display(len(data))
display(data.head())
print(len(set(data['ProductName'])))
print(len(set(data['ProductID'])))
print(data.dtypes)

68584

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


452
452
CustomerID      int64
FirstName      object
LastName       object
SalesID         int64
ProductID       int64
ProductName    object
Quantity        int64
dtype: object


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [52]:
col_clean = ['FirstName','LastName','SalesID','ProductID']
data_drop = data.drop(col_clean,axis=1)

In [53]:
data_product = data_drop.groupby(['CustomerID', 'ProductName'],sort=False).sum()
display(len(data_product))
data_product.head()

63628

,,Quantity
CustomerID,ProductName,
61288,Bread - Hot Dog Buns,16
77352,Bread - Hot Dog Buns,20
40094,Bread - Hot Dog Buns,11
23548,Bread - Hot Dog Buns,6
78981,Bread - Hot Dog Buns,20


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [58]:
data_pivot = data_product.pivot_table(index=["ProductName"],columns=["CustomerID"]).fillna(0)
data_pivot.head()

Quantity                                      \
CustomerID                       33    200   264   356   412   464   477     
ProductName                                                                  
Bread - Hot Dog Buns               0.0   0.0   0.0   0.0   0.0   0.0   1.0   
Cheese Cloth No 100                0.0   0.0   1.0   0.0   1.0   1.0   1.0   
Arizona - Green Tea                0.0   0.0   0.0   0.0   0.0   0.0   0.0   
Beer - Original Organic Lager      1.0   1.0   0.0   0.0   0.0   0.0   0.0   
Wine - Ruffino Chianti             0.0   1.0   0.0   0.0   0.0   0.0   0.0   

                                                 ...                           \
CustomerID                    639   649   669    ...  97697 97753 97769 97793   
ProductName                                      ...                            
Bread - Hot Dog Buns            0.0   0.0   0.0  ...    0.0   0.0  25.0   0.0   
Cheese Cloth No 100             0.0   0.0   0.0  ...    0.0   0.0  50.0   0.0   
Arizona - Green Tea             0.0   1.0   0.0  ...    0.0   0.0   0.0   0.0   
Beer - Original Organic Lager   0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   
Wine - Ruffino Chianti          0.0   0.0   0.0  ...    0.0   0.0   0.0   0.0   

                                                                   
CustomerID                    97900 97928 98069 98159 98185 98200  
ProductName                                                        
Bread - Hot Dog Buns            0.0   0.0   0.0   0.0  25.0   0.0  
Cheese Cloth No 100             0.0   0.0   0.0   0.0   0.0  25.0  
Arizona - Green Tea             0.0  25.0   0.0   0.0   0.0   0.0  
Beer - Original Organic Lager   0.0   0.0   0.0   0.0   0.0   0.0  
Wine - Ruffino Chianti          0.0  25.0  25.0   0.0   0.0   0.0  

[5 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [61]:
distances = pd.DataFrame(1/(1 + squareform(pdist(data_pivot.T, 'euclidean'))), 
                         index=data_pivot.columns, columns=data_pivot.columns)

distances.head()

Quantity                                          \
CustomerID              33        200       264       356       412     
         CustomerID                                                     
Quantity 33          1.000000  0.077421  0.087047  0.081800  0.080634   
         200         0.077421  1.000000  0.078448  0.076435  0.073693   
         264         0.087047  0.078448  1.000000  0.080070  0.081800   
         356         0.081800  0.076435  0.080070  1.000000  0.076435   
         412         0.080634  0.073693  0.081800  0.076435  1.000000   

                                                                       \
CustomerID              464       477       639       649       669     
         CustomerID                                                     
Quantity 33          0.082709  0.074573  0.083020  0.081503  0.080070   
         200         0.075255  0.075956  0.076435  0.077674  0.076923   
         264         0.080350  0.076923  0.080634  0.082100  0.078448   
         356         0.078187  0.075025  0.082403  0.077171  0.075956   
         412         0.078711  0.075025  0.082403  0.078187  0.078448   

                       ...                                             \
CustomerID             ...        97697     97753     97769     97793   
         CustomerID    ...                                              
Quantity 33            ...     0.004811  0.004669  0.004412  0.005019   
         200           ...     0.004824  0.004681  0.004431  0.005047   
         264           ...     0.004822  0.004674  0.004416  0.005035   
         356           ...     0.004816  0.004671  0.004416  0.005038   
         412           ...     0.004810  0.004702  0.004414  0.005034   

                                                                       \
CustomerID              97900     97928     98069     98159     98185   
         CustomerID                                                     
Quantity 33          0.004312  0.004515  0.004583  0.004355  0.004167   
         200         0.004311  0.004521  0.004614  0.004367  0.004166   
         264         0.004322  0.004543  0.004595  0.004365  0.004179   
         356         0.004310  0.004526  0.004578  0.004365  0.004175   
         412         0.004318  0.004530  0.004578  0.004367  0.004177   

                               
CustomerID              98200  
         CustomerID            
Quantity 33          0.004333  
         200         0.004335  
         264         0.004333  
         356         0.004339  
         412         0.004349  

[5 rows x 1000 columns]

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [78]:
#distances.idxmax()
similarities = distances.sort_index(axis=0, level=None)#[1:]
similarities

Quantity                                          \
CustomerID              33        200       264       356       412     
         CustomerID                                                     
Quantity 33          1.000000  0.077421  0.087047  0.081800  0.080634   
         200         0.077421  1.000000  0.078448  0.076435  0.073693   
         264         0.087047  0.078448  1.000000  0.080070  0.081800   
         356         0.081800  0.076435  0.080070  1.000000  0.076435   
         412         0.080634  0.073693  0.081800  0.076435  1.000000   
         464         0.082709  0.075255  0.080350  0.078187  0.078711   
         477         0.074573  0.075956  0.076923  0.075025  0.075025   
         639         0.083020  0.076435  0.080634  0.082403  0.082403   
         649         0.081503  0.077674  0.082100  0.077171  0.078187   
         669         0.080070  0.076923  0.078448  0.075956  0.078448   
         694         0.078977  0.075025  0.080070  0.076923  0.075956   
         756         0.083651  0.075956  0.082403  0.076435  0.078448   
         883         0.078977  0.076435  0.083651  0.082403  0.081210   
         891         0.076678  0.073478  0.077171  0.072637  0.076194   
         1008        0.083972  0.080350  0.088152  0.085297  0.083333   
         1034        0.080920  0.074798  0.081503  0.077171  0.074798   
         1066        0.079792  0.077171  0.079792  0.075720  0.075255   
         1072        0.085297  0.083972  0.087410  0.082709  0.082709   
         1336        0.081800  0.074573  0.077929  0.078448  0.078977   
         1428        0.075956  0.072844  0.076923  0.075025  0.074573   
         1435        0.076923  0.073265  0.075956  0.075025  0.075025   
         1534        0.083020  0.076923  0.082403  0.081210  0.081800   
         1577        0.084959  0.076923  0.081210  0.077421  0.076923   
         1594        0.078187  0.075720  0.081503  0.074798  0.076194   
         1754        0.082100  0.077674  0.079792  0.078187  0.077674   
         1839        0.076678  0.075720  0.076678  0.078711  0.076678   
         1920        0.081503  0.082100  0.082709  0.079792  0.076678   
         2187        0.082100  0.076194  0.079792  0.079246  0.078187   
         2329        0.077674  0.075720  0.076194  0.077171  0.077171   
         2503        0.085983  0.080350  0.082709  0.083333  0.080350   
...                       ...       ...       ...       ...       ...   
         95314       0.004431  0.004429  0.004410  0.004427  0.004431   
         95372       0.004687  0.004691  0.004677  0.004679  0.004676   
         95819       0.004476  0.004475  0.004467  0.004473  0.004471   
         96024       0.004551  0.004564  0.004553  0.004558  0.004577   
         96088       0.004248  0.004240  0.004236  0.004234  0.004230   
         96272       0.004450  0.004436  0.004439  0.004441  0.004456   
         96522       0.004948  0.004974  0.004972  0.004956  0.004978   
         96524       0.004205  0.004227  0.004211  0.004214  0.004220   
         96560       0.004215  0.004231  0.004209  0.004231  0.004222   
         96615       0.004531  0.004537  0.004517  0.004524  0.004523   
         96666       0.004943  0.004918  0.004916  0.004913  0.004918   
         96684       0.004435  0.004445  0.004444  0.004441  0.004454   
         97029       0.004802  0.004812  0.004816  0.004807  0.004804   
         97052       0.004655  0.004641  0.004642  0.004644  0.004652   
         97063       0.004471  0.004473  0.004474  0.004473  0.004471   
         97093       0.004975  0.004953  0.004963  0.004963  0.004968   
         97201       0.004779  0.004775  0.004790  0.004776  0.004784   
         97282       0.004478  0.004491  0.004467  0.004491  0.004478   
         97324       0.004032  0.004041  0.004045  0.004042  0.004035   
         97495       0.004782  0.004783  0.004768  0.004778  0.004789   
         97697       0.004811  0.004824  0.004822  0.004816  0.004810   
      

In [72]:
 print(list(distances))

[('Quantity', 33), ('Quantity', 200), ('Quantity', 264), ('Quantity', 356), ('Quantity', 412), ('Quantity', 464), ('Quantity', 477), ('Quantity', 639), ('Quantity', 649), ('Quantity', 669), ('Quantity', 694), ('Quantity', 756), ('Quantity', 883), ('Quantity', 891), ('Quantity', 1008), ('Quantity', 1034), ('Quantity', 1066), ('Quantity', 1072), ('Quantity', 1336), ('Quantity', 1428), ('Quantity', 1435), ('Quantity', 1534), ('Quantity', 1577), ('Quantity', 1594), ('Quantity', 1754), ('Quantity', 1839), ('Quantity', 1920), ('Quantity', 2187), ('Quantity', 2329), ('Quantity', 2503), ('Quantity', 2556), ('Quantity', 2566), ('Quantity', 2582), ('Quantity', 2617), ('Quantity', 2686), ('Quantity', 2754), ('Quantity', 2776), ('Quantity', 2902), ('Quantity', 2915), ('Quantity', 2939), ('Quantity', 3074), ('Quantity', 3253), ('Quantity', 3267), ('Quantity', 3305), ('Quantity', 3317), ('Quantity', 3472), ('Quantity', 3531), ('Quantity', 3535), ('Quantity', 3544), ('Quantity', 3885), ('Quantity', 3

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.